# Explore here

In [95]:
# Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


In [96]:
df = pd.read_csv('/workspaces/pauline-Proyecto-Naive-Bayes/data/raw/playstore_reviews.csv')
#Como dicho en las instrucciones, eliminamos package_name, ya que de eso no depende la clasificación de la reseña
df.drop(['package_name'], axis=1, inplace=True)
df.head()

,review,polarity
0,privacy at least put some option appear offli...,0
1,"messenger issues ever since the last update, ...",0
2,profile any time my wife or anybody has more ...,0
3,the new features suck for those of us who don...,0
4,forced reload on uploading pic on replying co...,0


In [97]:
#Quiero saber cuantas filas hay :
print(df.shape)
#Tipo de data y check si hay nulos/faltantes
print(df.info())
# Revisamos que no haya fila duplicada:
sum(df.duplicated())

(891, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   review    891 non-null    object
 1   polarity  891 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.0+ KB
None


0

In [98]:
#Eliminar espacios y convertir a minúsculas el texto: 
df["review"] = df["review"].str.strip().str.lower()

#Y dividimos los datos en conjuntos de entrenamiento y testeo
X = df['review']
y = df[['polarity']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [99]:
X_train.head()

331    just did the latest update on viber and yet ag...
733    keeps crashing it only works well in extreme d...
382    the fail boat has arrived the 6.0 version is t...
704    superfast, just as i remember it ! opera mini ...
813    installed and immediately deleted this crap i ...
Name: review, dtype: object

In [100]:
vectorizer = CountVectorizer(stop_words = "english") #ese de stop_word se usa para eliminar las palabras más comunes y sin valor informativo
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)



In [101]:
#Chequeamos con algunas palabras o características que han sido quitadas:
vectorizer.get_feature_names_out()


array(['000', '04', '0x', ..., 'žŕ', 'žŕľ', 'ˇŕ'],
      shape=(3310,), dtype=object)

In [102]:
X_test_vec.toarray()[1]

array([0, 0, 0, ..., 0, 0, 0], shape=(3310,))

In [103]:
#Mostrar una review de prueba (texto)
print(f'Review de prueba: {X_test.iloc[1]}')


print('\nPalabras del conjunto de entrenamiento que también aparecen en la review de test junto con su aparición:\n')
for i, cont in enumerate(X_test_vec.toarray()[1]):
  if cont!=0:
    print(f'Palabra: "{vectorizer.get_feature_names_out()[i]}"')
    print(f'--> Aparece {cont} veces en la review.')


Review de prueba: whatsapp i use this app now that blackberry messenger has basically gone away. my friends & family live all over the world. this really helps keep us in touch!

Palabras del conjunto de entrenamiento que también aparecen en la review de test junto con su aparición:

Palabra: "app"
--> Aparece 1 veces en la review.
Palabra: "away"
--> Aparece 1 veces en la review.
Palabra: "basically"
--> Aparece 1 veces en la review.
Palabra: "family"
--> Aparece 1 veces en la review.
Palabra: "friends"
--> Aparece 1 veces en la review.
Palabra: "gone"
--> Aparece 1 veces en la review.
Palabra: "helps"
--> Aparece 1 veces en la review.
Palabra: "live"
--> Aparece 1 veces en la review.
Palabra: "messenger"
--> Aparece 1 veces en la review.
Palabra: "really"
--> Aparece 1 veces en la review.
Palabra: "touch"
--> Aparece 1 veces en la review.
Palabra: "use"
--> Aparece 1 veces en la review.
Palabra: "whatsapp"
--> Aparece 1 veces en la review.
Palabra: "world"
--> Aparece 1 veces en la r

**Paso 3: Construye un naive bayes.** Elegimos **Multinomial** porque variable predictora es categorica, con varias posibilidades.   (Gauss es para regresion y Bernoulli cuando las variables son binarias).

In [104]:
#Inicializar y entrenar el clasificador Naive Bayes Multinomial
clf_multi = MultinomialNB().fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = clf_multi.predict(X_test_vec)




# Evaluar el rendimiento del modelo
print(f'Evaluación del modelo: {classification_report(y_test, y_pred)}\n')



Evaluación del modelo:               precision    recall  f1-score   support

           0       0.84      0.90      0.87       126
           1       0.73      0.60      0.66        53

    accuracy                           0.82       179
   macro avg       0.79      0.75      0.77       179
weighted avg       0.81      0.82      0.81       179




/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


- **Accuracy** (=exactitud) --> alta con 82%

**Precision** :
- Clase 0 (negativa): 0.84 --> de todas las veces que el modelo predijo que una muestra pertenecía a la clase 0, un 84% de esas veces realmente era 0.
- Clase 1 (positiva): 0.73 --> de todas las veces que el modelo predijo que una muestra pertenecía a la clase 1, un 73% de esas veces realmente era 1.

**Recall** : sensibilidad
- Clase 0 (negativa): 0.90. El modelo identificó correctamente el 90% de todas las instancias que realmente pertenecen a la clase 0.
- Clase 1 (positiva): 0.60. Solo el 60% de las instancias que realmente pertenecen a la clase 1 fueron correctamente identificadas por el modelo.

**F1** = Recall vs precion :

- Clase 0 (negativa): 0.87. Esta es una puntuación bastante buena, lo que indica que la precisión y el recall de la clase 0 están equilibrados y son bastante altos.
- Clase 1 (positiva): 0.66. Más de 20 puntos más bajo, el modelo tiene más dificultades con la clase de comentarios positivos.


El modelo tiene un buen rendimiento para comentarios negativos (alta precisión y recall), pero más dificultades a predecir comentarios positivos (baja recall y precisión comparado con 0).



Testeamos los 2 otros
- **GAUSS**:


In [105]:
#probamos con los 2 otros : 

#Inicializar y entrenar el clasificador Naive GaussianNB
clf_gauss = GaussianNB().fit(X_train_vec.toarray(), y_train)

#Realizar predicciones en el conjunto de prueba
y_pred_gauss = clf_gauss.predict(X_test_vec.toarray())


#Evaluar el rendimiento del modelo
print(f'Evaluación del modelo: {classification_report(y_test, y_pred_gauss)}\n')



Evaluación del modelo:               precision    recall  f1-score   support

           0       0.85      0.88      0.86       126
           1       0.69      0.62      0.65        53

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.80      0.80      0.80       179




/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Resultados bastante similares que con m ultinomial

 **Accuracy** (=exactitud) --> alta con 80%

**Precision**:
- Clase 0 (negativa): 0.85 --> de todas las veces que el modelo predijo que una muestra pertenecía a la clase 0, un 85% de esas veces realmente era 0.
- Clase 1 (positiva): 0.69 --> de todas las veces que el modelo predijo que una muestra pertenecía a la clase 1, un 69% de esas veces realmente era 1.

**Recall**: sensibilidad
- Clase 0 (negativa): 0.88. El modelo identificó correctamente el 88% de todas las instancias que realmente pertenecen a la clase 0.
- Clase 1 (positiva): 0.62. Solo el 62% de las instancias que realmente pertenecen a la clase 1 fueron correctamente identificadas por el modelo.

**F1** = Recall vs precisión :

- Clase 0 (negativa): 0.86. precisión y el recall de la clase 0 están equilibrados y son bastante altos.
- Clase 1 (positiva): 0.65. Igual, más de 20 puntos más bajo, el modelo tiene más dificultades con la clase de comentarios positivos.

El modelo tiene un buen rendimiento para comentarios negativos (alta precisión y recall), pero más dificultades a predecir comentarios positivos (baja recall y precisión comparado con 0).

- BERNOUILLI

In [106]:
#Inicializar y entrenar el clasificador Naive BernoulliNB
clf_ber = BernoulliNB()
clf_ber.fit(X_train_vec.toarray(), y_train)

#Realizar predicciones en el conjunto de prueba
y_pred_ber = clf_ber.predict(X_test_vec.toarray())

#Evaluar el rendimiento del modelo
print(f'Evaluación del modelo: {classification_report(y_test, y_pred_ber)}\n')



Evaluación del modelo:               precision    recall  f1-score   support

           0       0.79      0.93      0.85       126
           1       0.70      0.40      0.51        53

    accuracy                           0.77       179
   macro avg       0.74      0.66      0.68       179
weighted avg       0.76      0.77      0.75       179




/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



**Accuracy** (=exactitud) --> 77%, lo que es más bajo que el modelo Gaussiano y también inferior al modelo base.

**Precision:**
- Clase 0 (negativa): 0.79 --> De todas las veces que el modelo predijo que una muestra pertenecía a la clase 0, un 79% de esas veces realmente era 0.
- Clase 1 (positiva): 0.70 --> De todas las veces que el modelo predijo que una muestra pertenecía a la clase 1, un 70% de esas veces realmente era 1.

**Recall**: sensibilidad

- Clase 0 (negativa): 0.93. El modelo identificó correctamente el 93% de todas las instancias que realmente pertenecen a la clase 0.
- Clase 1 (positiva): 0.40. Solo el 40% de las instancias que realmente pertenecen a la clase 1 fueron correctamente identificadas por el modelo.

**F1** = Recall vs precisión:
- Clase 0 (negativa): 0.85. Al igual que el modelo Gaussiano, el modelo Bernoulli tiene un buen equilibrio entre precisión y recall para la clase 0.
- Clase 1 (positiva): 0.51. Mucho más baja =  dificultades del modelo para clasificar correctamente los comentarios positivos.

Todos los indicatores son más bajos que los modelos anteriores, sobre todo por los comentarios negativos, solo 40% de sensibilidad u 0,51 F1. indica que no es muy fiable,



**Conclusion sobre comparación de los 3 modelos**:

El *mejor modelo es el multinonmial*, que hemos hecho primer : 
- mejor accuracy
- % más alto para precision, recall y F1
- promedio ponderado (miramos mejor eso porque el conjunto esta bastante desbalanceado) también mejor más altos en todos los casos, que con Gauss o Bernoulli.

Paso 4: Optimización del modelo multinomial :

In [110]:
#Alpha = suavización = principal hiperparametro de Bayes Multinomial, vamos probar con varias valores y usando GridSearchCV:

param_grid = {'alpha': [0.1, 0.5, 1, 2, 5, 10],  #Valores comunes para alpha
              } 

#Clasificador:
nb = MultinomialNB()

#GridSearchCV para buscar el mejor valor de alpha y fit_prior
grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)


#Ajustar el modelo con los datos de entrenamiento
grid_search.fit(X_train_vec, y_train)

print(f'La mejor valor para alpha es: {grid_search.best_params_}')





Fitting 5 folds for each of 6 candidates, totalling 30 fits
La mejor valor para alpha es: {'alpha': 0.5}


In [111]:
#Evaluar el modelo con el mejor alpha encontrado
y_pred_optimized = grid_search.best_estimator_.predict(X_test_vec)
print(f'Evaluación del modelo con GridSearchCV y optimización de hiperparámetros:')
print(classification_report(y_test, y_pred_optimized))

Evaluación del modelo con GridSearchCV y optimización de hiperparámetros:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       126
           1       0.73      0.66      0.69        53

    accuracy                           0.83       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.82      0.83      0.82       179



In [ ]:
#VS volvemos a poner evaluación del modelo antes de optimisación :               
#                precision    recall  f1-score   support

#           0       0.84      0.90      0.87       126
#           1       0.73      0.60      0.66        53

#    accuracy                           0.82       179
#   macro avg       0.79      0.75      0.77       179
#weighted avg       0.81      0.82      0.81       179



**El model mejoró** : 
Accuracy subó a 83 % (+ 1 punto) y todos los resultados son = o mejores que antes de la optimización

In [119]:


#Guardar el modelo en un archivo
import joblib
joblib.dump(grid_search.best_estimator_, '/workspaces/pauline-Proyecto-Naive-Bayes/models/modelo_multinomial.pkl')


['/workspaces/pauline-Proyecto-Naive-Bayes/models/modelo_multinomial.pkl']